# Итоговый проект на тему "Отслеживание русского языка жестов"
Модель представляет собой программу по отслеживанию языка жестов при помощи видеокамеры,
данные собирались во время использования модели самой программой.

Обучение модели происходило на сайте: https://teachablemachine.withgoogle.com/train/image

Используемые версии tensorflow и keras: 2.7.0

In [32]:
# установка и импорт библиотек
# !pip install cvzone
# !pip install tensorflow==2.7.0
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math

# Создание класса захвата видео с камеры
cap = cv2.VideoCapture(0)
# Создание класса распознавания рук (ограничение в 1 руку)
detector = HandDetector(maxHands=1)

# Константы отступа для изображения руки и размера изображения
offset = 20
imgSize = 256

# Константы папки с картинками для модели и обнуление счетчика кадров и ошибок
folder = "data/18"
counter = 0
warn = 0

while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

        imgCropShape = imgCrop.shape

#       Соотношение ширины к длине изображения кисти
        aspectRatio = h / w

#       Подстройка изображения под заданные размеры, чтобы оно умещалось в окно на белом фоне
#       Использовалось для обучения модели и распознавания жестов
#       конструкция try используется для предотвращения вылета программы
#       при выходе за края области отслеживания
        try:
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize

            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
        except:
            warn += 1
            if warn == 15:
                print('За областью отслеживания!')
                warn = 0
            pass
        cv2.imshow("ImageWhite", imgWhite)

    cv2.imshow("Image", img)

#   При зажатии 'S' программа будет сохранять изображения руки в папку с data
#   Директории в которой расположен блокнот, использовалась для обучения модели
    key = cv2.waitKey(1)
    if key == ord("s"):
#       Счетчик сделанных кадров
        counter += 1
        cv2.imwrite(f'{folder}/{counter}.jpg', imgWhite)
        print(counter)
        
#   Кнопка Q завершает отслеживание и закрывает окна  
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [44]:
import tensorflow
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
 
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier("Model/keras_model.h5", "Model/labels.txt")
 
offset = 20
imgSize = 300
labels = ['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'С', 'Ы', ' ', 'И', "Й", "К", "Л", "Н", "М", "О", "П"]
text = ''
warn = 0
frameNum =0
while True:
    success, img = cap.read()
#   Копирование изначального изображения с камеры, чтобы не отправить видимые соединения для предикта модели
#   Линии и точки будут рисоваться на 
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
        imgCropShape = imgCrop.shape
 
        aspectRatio = h / w
        try:
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
                prediction, index = classifier.getPrediction(imgWhite, draw=False)
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
                prediction, index = classifier.getPrediction(imgWhite, draw=False)
        except:
            if warn == 15:
                print('За областью отслеживания!')
                warn = 0
            
#       Обвод кисти в прямоугольник и отображение найденной буквы
        cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)
        
#       Если жест не изменялся 45 кадров, запоминает его как введенный
    if key == ord('f'):
        if labels[index] and labels[index] == labels[index]:
            if frameNum < 10:
                frameNum += 1
            else:
                frameNum = 0
                text = text + str(labels[index])
                print(labels[index])        

#       Можно раскомментировать стору, чтобы увидеть как модель классифицирует кисть
    cv2.imshow("ImageWhite", imgWhite)
    cv2.imshow("Image", imgOutput)
    key = cv2.waitKey(1)
#   Кнопка Q завершает отслеживание и закрывает окна  
    if key == ord('q'):
        break
print(text)
cap.release()
cv2.destroyAllWindows()

В
В
